# Historical Market Returns

For this project, we are replicating the excess market return outlined on the French Fama website in the below URL:

https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

This is a value-weighted portfolio calculating the total excess US market return from July 1926 to January 2024 (subject to available data). Stocks are analyzed on a monthly basis. The risk free rate is imported from the French Fama website. We filter our stocks to only use the 10 and 11 sharecodes and the 1, 2 and 3 exchanges, per the French Fama website.

I used the below libraries for this project.

In [2]:
import wrds
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pandas_datareader
import datetime
import numpy as np

Below is our code for connecting to our CRSP data using the WRDS library. We are downloading monthly stock data and monthly delisted stock data for all available stocks that meet the French Fama requirements.

In [3]:
id = 'johncrowe'
conn = wrds.Connection(id)

# Download monthly crsp stock data
a = conn.raw_sql("""
                    select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                    a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr
                    from crspq.msf as a 
                    left join crsp.msenames as b
                    on a.permno=b.permno
                    and b.namedt<=a.date
                    and a.date<=b.nameendt
                    where a.date between '01/01/1900' and '12/31/2023'
                    """)

# Download monthly crsp delisted stock data
b = conn.raw_sql("""
                    select permno, dlret, dlstdt, dlstcd
                    from crspq.msedelist
                    """)

c = conn.get_table(library='ff', table='factors_monthly')

conn.close()

Enter your WRDS username [John]:johncrowe
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


Below we create new dataframes so that we do not have to reconnect to the SQL servers to rerun our code.

In [4]:
crsp_raw = a.copy()
dlret_raw = b.copy()
french = c.copy()

Below we are importing the monthly risk free rate.

In [5]:
# Here is an alternative way using wrds package (FF3 factors only...)
french = french[['date','mktrf','smb','hml','rf']]
french['mkt'] = french['mktrf'] + french['rf']
french['date'] = pd.to_datetime(french['date']) + MonthEnd(0)

# Change columns to floats
french['mktrf'] = french['mktrf'].astype(float)
french['smb'] = french['smb'].astype(float)
french['hml'] = french['hml'].astype(float)
french['rf'] = french['rf'].astype(float)

# Create year and month columns for french data
french['year'] = french['date'].dt.year
french['month'] = french['date'].dt.month

Here we use some data cleaning and rearraning. First we arrange by date and permno, then we manipulate the date portion so that we can ensure they are in the proper order and then merge delisted with crsp to create our total return column, which we will call 'ret'. We then attach the lagged market return and end up with a row in our dataframe for each stock-year-month.

In [6]:
# Clean crsp_raw data
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop=True).copy()
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%y-%m-%d') + MonthEnd(0)
crsp_raw['prc'] = np.absolute(crsp_raw['prc'])

# Clean dlret_raw data
dlret_raw = dlret_raw.sort_values(['permno', 'dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date'] = dlret_raw['dlstdt'] + MonthEnd(0)

# Merge crsp_raw with dlret_raw
stocks = crsp_raw.merge(dlret_raw[['permno','date','dlret']], how='outer', on=['permno','date'])
stocks['dlret'] = stocks['dlret'].fillna(0)
stocks['shrcd'] = stocks['shrcd'].ffill()
stocks['exchcd'] = stocks['exchcd'].ffill()
stocks['ret'] = stocks['ret'].fillna(0)
stocks['prc'] = stocks['prc'].fillna(0)
stocks['shrout'] = stocks['shrout'].fillna(0)
stocks['me'] = stocks['prc']*stocks['shrout']

# Redefine 'ret' to include delisted return
stocks['ret'] = (1 + stocks['ret']) * (1 + stocks['dlret']) - 1

# Attach Lagged Market Equity (to be used as weights) for each stock-year-month
stocks = stocks.sort_values(by=['permno','date']).reset_index().drop('index',axis=1).copy()
stocks['daten'] = stocks['date'].dt.year*12 + stocks['date'].dt.month
stocks['IsValidLag'] = stocks['daten'].diff(1) == 1 # Lag date has to be the lagged date
stocks.loc[stocks[stocks['permno'].diff(1) != 0].index,['IsValidLag']] = False # Lagged date has to be the same security
stocks['lme'] = stocks[['permno','me']].groupby('permno').shift(1)
stocks.loc[stocks[stocks['IsValidLag'] == False].index,['lme']] = np.nan
stocks = stocks.drop(['IsValidLag','daten'], axis=1)

# Filter for 'shrcd' = 10 or 11 and 'exchcd' = 1, 2 or 3 per the Fama French website
stocks = stocks[((stocks['shrcd'] == 10) | (stocks['shrcd'] == 11)) & ((stocks['exchcd'] == 1) | (stocks['exchcd'] == 2) | (stocks['exchcd'] == 3))].copy()

# Create year and month columns for readability
stocks['year'] = stocks['date'].dt.year
stocks['month'] = stocks['date'].dt.month

# Rearrange columns into desired format
stocks = stocks[['permno', 'year', 'month', 'prc', 'shrout', 'lme', 'ret']]

# Drop NAs so that columns without lagged market equity value are not included
stocks = stocks.dropna()

Now we have our stocks in the desired stock-year-month with lagged market price and returns. We can now filter through all available times and find the total market return for each month. The resulting dataframe will be in year-month format.

In [7]:
# Create our lists for year-month dataframe
years = []
months = []
value_monthly_returns = []
equal_monthly_returns = []
lagged_market_values = []
    
# Find all unique year-month combinations present in both stocks and french
french_date = french[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
stock_date = stocks[['year', 'month']].drop_duplicates().sort_values(by=['year', 'month'])
date = french_date.merge(stock_date, how='inner', on=['year', 'month'])

# Iterate over the DataFrame and print year and month
for index, row in date.iterrows():
    # Find monthly returns
    year = row['year']
    month = row['month']
    this_stocks = stocks[(stocks['year'] == year) & (stocks['month'] == month)]
    rf = french[(french['year'] == year) & (french['month'] == month)]['rf'].iloc[0]
    start_value = this_stocks['lme'].sum()
    end_value = (this_stocks['lme']*(1 + this_stocks['ret'])).sum()
    value_monthly_return = (end_value - start_value)/start_value - rf
    equal_monthly_return = this_stocks['ret'].mean() - rf
    
    # Append results
    years.append(year)
    months.append(month)
    value_monthly_returns.append(value_monthly_return)
    equal_monthly_returns.append(equal_monthly_return)
    lagged_market_values.append(start_value)

data = {'year': years,
        'month': months,
        'MktRF-VW': value_monthly_returns,
        'MktRF-EW' : equal_monthly_returns,
        'Lagged Mkt Value': lagged_market_values}

monthly_returns = pd.DataFrame(data)

Using the risk-free rate of return from French’s website, we will report the market excess returns for French time series and out replicated time series comparing the following factors: annualized return, annualized volatility, annualized Sharpe ratio, skewness, and excess kurtosis. We are using market data from July 1926 to January 2024.

In [8]:
# Merge French Fama and replication so that we use the same timeframe
both = monthly_returns.merge(french, how = 'inner', on = ['year', 'month'])

# Calculate French Fama metrics
french_mean = 1200*both['mktrf'].mean()  
french_std = 100*np.sqrt(12)*both['mktrf'].std() 
french_SR = french_mean/french_std
french_skew = both['mktrf'].skew()
french_kurtosis = both['mktrf'].kurtosis()

# Calculate replicated metrics
replicated_mean = 1200*both['MktRF-VW'].mean()  # Calculate the mean of the 'MktRF' column
replicated_std = 100*np.sqrt(12)*both['MktRF-VW'].std() 
replicated_SR = replicated_mean/replicated_std
replicated_skew = both['MktRF-VW'].skew()
replicated_kurtosis = both['MktRF-VW'].kurtosis()

# Convert floats to strings for formatting
french_mean, french_std, french_SR, french_skew, french_kurtosis = '{:.2f}'.format(french_mean), '{:.2f}'.format(french_std), '{:.2f}'.format(french_SR), '{:.2f}'.format(french_skew), '{:.2f}'.format(french_kurtosis)
replicated_mean, replicated_std, replicated_SR, replicated_skew, replicated_kurtosis = '{:.2f}'.format(replicated_mean), '{:.2f}'.format(replicated_std), '{:.2f}'.format(replicated_SR), '{:.2f}'.format(replicated_skew), '{:.2f}'.format(replicated_kurtosis)

# Create final dataframe and output results
output1 = pd.DataFrame({'Variable Name': ['Mean', 'Volatility', 'Sharpe Ratio', 'Skewness', 'Kurtosis'], 
                        'Fama': [french_mean, french_std, french_SR, french_skew, french_kurtosis],
                        'Replication': [replicated_mean, replicated_std, replicated_SR, replicated_skew, replicated_kurtosis]})

output1.style.hide()

Variable Name,Fama,Replication
Mean,8.14,8.13
Volatility,18.51,18.47
Sharpe Ratio,0.44,0.44
Skewness,0.16,0.17
Kurtosis,7.42,7.38


Now I can calculate the correlation between my replicated time series and French Fama’s time series as well as the maximum absolute difference between the two time series. 

As you can see below, the correlation between the two sets is very close to one. This makes sense since we attempted to replicate the French Fama market returns and expect a high correlation. The absolute max value difference is very small, and we can consider any difference between these two sets to be economically negligible. The difference is not zero, however, because French and Fama likely used certain methods not addressed on their website.

In [9]:
# Find max difference and correlation between French Fama and replicated time series
max_absolute_diff = (both['MktRF-VW'] - both['mktrf']).abs().max()
correlation = both['MktRF-VW'].corr(both['mktrf'])

# Convert floats to strings for formatting
max_absolute_diff, correlation = '{:.4f}'.format(max_absolute_diff), '{:.4f}'.format(correlation)

#Create final dataframe and output results
output2 = pd.DataFrame({'Variable Name': ['Correlation', 'Max Absolute Value Difference'], 
                        'Value': [correlation, max_absolute_diff]})

output2.style.hide()

Variable Name,Value
Correlation,0.9998
Max Absolute Value Difference,0.0350
